In [15]:
### Prompt Templates
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate 

chat = ChatOpenAI(temperature=0.1) ## call Chat large language models API.

# Load a prompt template from a template.
template = PromptTemplate.from_template('{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')

prompt = template.format(
country_a='korea',
country_b='japan')

# Pass a single string input to the model and return a string prediction.
chat.predict(prompt) 

# Create a chat prompt template from a variety of message formats.
template = ChatPromptTemplate.from_messages ([
    ('system', 'you are a geography expert. and you only reply in {language}.'),
    ('ai', '무엇이 궁금하신가요? 저는 {name}입니다!'),
    ('human', '{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')
])

prompt = template.format_messages(language="korean", name='배다롱', country_a='한국', country_b='일본')

#Pass a message sequence to the model and return a message prediction.
chat.predict_messages(prompt) 

AIMessage(content='한국과 일본은 두 나라 모두 동아시아 지역에 위치하고 있습니다. 한국은 한반도의 남쪽에 위치하고 있으며, 일본은 일본 제도군으로 이루어진 섬나라입니다. 두 나라는 지형적으로 다양한 지형을 가지고 있으며, 한국은 산악 지형이 많고 일본은 화산 활동이 활발한 섬나라로 알려져 있습니다. 또한, 한국은 대륙과 인접해 있어 대륙적 영향을 받았고, 일본은 바다를 중심으로 발전해 왔습니다. 이러한 지리적 차이로 인해 두 나라의 문화, 역사, 경제 등이 다양한 면에서 차이를 보이고 있습니다.')

In [13]:
### Predict Messages
# from langchain.llms.openai import OpenAI
# llm = OpenAI(model_name="gpt-3.5-turbo-1106")
# a = llm.predict("How many planets are there?")
from  langchain.schema  import  HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

messages  = [
    SystemMessage(content='you are a geography expert. and you only reply in korean.'),
    AIMessage(content='무엇이 궁금하신가요? 저는 지리학 박사입니다!'),
    HumanMessage(content='한국과 일본의 지리적 차이에 대해 알려줘.')
]

# Pass a message sequence to the model and return a message prediction.
chat.predict_messages(messages)

AIMessage(content='한국과 일본은 지리적으로 많은 차이가 있습니다. 한국은 한반도에 위치하고 있으며, 주변에는 동해와 서해가 있습니다. 일본은 일본 제도로 이루어져 있으며, 주변에는 동중국해와 태평양이 있습니다. 또한, 한국은 산악 지형이 많고, 일본은 화산 활동이 활발한 섬나라로 알려져 있습니다. 이러한 지리적 차이로 두 나라의 자연환경과 문화가 다양하게 발전하였습니다.')